# Training a model with a GravyFlow Dataset

In this notebook we will use our generated dataset to train a keras model. We start with the needed imports: 

In [1]:
# Built-in imports
from typing import List, Dict
from pathlib import Path

# Import the GravyFlow module.
import gravyflow as gf

# Dependency imports: 
import numpy as np
import keras
from keras import ops
import jax.numpy as jnp
from keras.layers import Input, Permute, Conv1D, MaxPooling1D, Dense, Flatten, Dropout, ELU
from keras.models import Model

## Creating a TensorFlow dataset through composition:

Rather than generating a generic Python iterator, we can also use GravyFlow to create a custom TensorFlow dataset. This will give us the ability to utalise all the functionality provided by the TensorFlow dataset class, including seamless integration with keras models, whilst maintaining the ability to generate datasets quickly enough for real time training, only caching downloaded data segments.

First we will create a keras model, inspired by a model from the literature, found at Gabbard _et at._ here: https://link.aps.org/doi/10.1103/PhysRevLett.120.141103:

In [2]:
def create_gabbard(
        input_shape_onsource : int, 
        input_shape_offsource : int
    ) -> keras.Model:
    
    # Define the inputs based on the dictionary keys and expected shapes
    # Replace `input_shape_onsource` and `input_shape_offsource` with the actual shapes
    onsource_input = Input(shape=input_shape_onsource, name="ONSOURCE")
    offsource_input = Input(shape=input_shape_offsource, name="OFFSOURCE")

    # Pass the inputs to your custom Whiten layer
    # Assuming your Whiten layer can handle multiple inputs
    whiten_output = gf.Whiten()([onsource_input, offsource_input])

    x = Permute((2, 1))(whiten_output)
    
    # Convolutional and Pooling layers
    x = Conv1D(8, 64, padding='valid', name="Convolutional_1")(x)
    x = ELU(name="ELU_1")(x)
    x = MaxPooling1D(pool_size=4, strides=4, name="Pooling_1", padding="valid")(x)
    
    x = Conv1D(8, 32, padding='valid', name="Convolutional_2")(x)
    x = ELU(name="ELU_2")(x)
    x = Conv1D(16, 32, padding='valid', name="Convolutional_3")(x)
    x = ELU(name="ELU_3")(x)
    x = MaxPooling1D(pool_size=4, strides=4, name="Pooling_3", padding="valid")(x)
    
    # Flatten layer
    x = Flatten(name="Flatten")(x)
    
    # Dense layers with dropout
    x = Dense(64, name="Dense_1")(x)
    x = ELU(name="ELU_7")(x)
    x = Dropout(0.5, name="Dropout_1")(x)
    
    x = Dense(64, name="Dense_2")(x)
    x = ELU(name="ELU_8")(x)
    x = Dropout(0.5, name="Dropout_2")(x)
    
    outputs = Dense(1, activation='sigmoid', name=gf.ReturnVariables.INJECTION_MASKS.name)(x)
    
    # Create model
    model = Model(
        inputs=[onsource_input, offsource_input], 
        outputs={gf.ReturnVariables.INJECTION_MASKS.name: outputs}, 
        name="custom"
    )
    
    return model

Because we are only using one injection, we expect our output label to be a single value for each example, therefore we must adjust the dimensionality of the injection masks output with tensorflow datasets mapping functionality, we define the function we want to map to the dataset here:

In [3]:
class AdapterDataset(keras.utils.PyDataset):
    def __init__(self, dataset):
        super().__init__(workers=dataset.workers, use_multiprocessing=dataset.use_multiprocessing)
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        features, labels = self.dataset[index]
        if 'INJECTION_MASKS' in labels:
            mask = labels['INJECTION_MASKS']
            # Check if already processed: (Batch, 1)
            if len(mask.shape) == 2 and mask.shape[1] == 1:
                pass
            elif len(mask.shape) == 2: # (NumGenerators, Batch)
                mask = mask[0]
                mask = ops.expand_dims(mask, axis=-1)
            elif len(mask.shape) == 3: # (NumGenerators, Batch, Time)
                mask = mask[0]
                mask = jnp.max(mask, axis=-1)
                mask = ops.expand_dims(mask, axis=-1)
            
            labels['INJECTION_MASKS'] = mask
        return features, labels


TensorFlow and keras requires that the model and training dataset are created in the same scope, and is quite strict about these limitations. Thus we will here create our dataset and our model in the same scope. Nominally, it is anticipated that GravyFlow will mostly be used in Python scripts, rather than notebooks, where this will not be a problem if everything is kept in the same TensorFlow strategy:

In [4]:
examples_per_epoch = int(1E5)
num_validation_examples = int(1E4)
num_testing_examples = int(1E4)
steps_per_epoch = examples_per_epoch // gf.Defaults.num_examples_per_batch
validation_steps = num_validation_examples // gf.Defaults.num_examples_per_batch
testing_steps = num_testing_examples // gf.Defaults.num_examples_per_batch

# This object will be used to obtain real interferometer data based on specified parameters.
ifo_data_obtainer : gf.IFODataObtainer = gf.IFODataObtainer(
    observing_runs=gf.ObservingRun.O3, # Specify the observing run (e.g., O3).
    data_quality=gf.DataQuality.BEST,  # Choose the quality of the data (e.g., BEST).
    data_labels=[                      # Define the types of data to include.
        gf.DataLabel.NOISE, 
        gf.DataLabel.GLITCHES
    ],
    segment_order=gf.SegmentOrder.RANDOM, # Order of segment retrieval (e.g., RANDOM).
    force_acquisition=True,               # Force the acquisition of new data.
    cache_segments=False                  # Choose not to cache the segments.
)

# Initialize the noise generator wrapper:
# This wrapper will use the ifo_data_obtainer to generate real noise based on the specified parameters.
noise: gf.NoiseObtainer = gf.NoiseObtainer(
    ifo_data_obtainer=ifo_data_obtainer, # Use the previously set up IFODataObtainer object.
    noise_type=gf.NoiseType.REAL,        # Specify the type of noise as REAL.
    ifos=gf.IFO.L1                       # Specify the interferometer (e.g., LIGO Livingston L1).
)

scaling_method : gf.ScalingMethod = gf.ScalingMethod(
    value=gf.Distribution(
        min_=8.0,
        max_=15.0,
        type_=gf.DistributionType.UNIFORM
    ),
    type_=gf.ScalingTypes.SNR
)

# Define a uniform distribution for the mass of the first object in solar masses.
mass_1_distribution_msun : gf.Distribution = gf.Distribution(
    min_=10.0, 
    max_=60.0, 
    type_=gf.DistributionType.UNIFORM
)

# Define a uniform distribution for the mass of the second object in solar masses.
mass_2_distribution_msun : gf.Distribution = gf.Distribution(
    min_=10.0, 
    max_=60.0, 
    type_=gf.DistributionType.UNIFORM
)

# Define a uniform distribution for the inclination of the binary system in radians.
inclination_distribution_radians : gf.Distribution = gf.Distribution(
    min_=0.0, 
    max_=np.pi, 
    type_=gf.DistributionType.UNIFORM
)

# Initialize a PhenomD waveform generator with the defined distributions.
# This generator will produce waveforms with randomly varied masses and inclination angles.
phenom_d_generator : gf.WaveformGenerator = gf.CBCGenerator(
    mass_1_msun=mass_1_distribution_msun,
    mass_2_msun=mass_2_distribution_msun,
    inclination_radians=inclination_distribution_radians,
    scaling_method=scaling_method,
    injection_chance=0.5 # Set so half produced examples will not contain this signal
)

training_dataset  = gf.Dataset(       
    noise_obtainer=noise,
    waveform_generators=phenom_d_generator,
    input_variables=[
        gf.ReturnVariables.ONSOURCE, 
        gf.ReturnVariables.OFFSOURCE, 
    ],
    output_variables=[
        gf.ReturnVariables.INJECTION_MASKS
    ]
)

validation_dataset  = gf.Dataset(       
    noise_obtainer=noise,
    waveform_generators=phenom_d_generator,
    seed=1001, # Implement different seed to generate different waveforms,
    group="validate", # Ensure noise is pulled from those marked for validation.
    input_variables=[
        gf.ReturnVariables.ONSOURCE, 
        gf.ReturnVariables.OFFSOURCE, 
    ],
    output_variables=[
        gf.ReturnVariables.INJECTION_MASKS
    ]
)

testing_dataset  = gf.Dataset(     
    num_examples_per_batch = 32,  
    noise_obtainer=noise,
    waveform_generators=phenom_d_generator,
    seed=1002, # Implement different seed to generate different waveforms,
    group="test", # Ensure noise is pulled from those marked for validation.
    input_variables=[
        gf.ReturnVariables.ONSOURCE, 
        gf.ReturnVariables.OFFSOURCE, 
    ],
    output_variables=[
        gf.ReturnVariables.INJECTION_MASKS
    ]
)

for input_example, _ in [training_dataset[0]]:
    input_shape_onsource = input_example["ONSOURCE"].shape[1:]  # Exclude batch dimension    
    input_shape_offsource = input_example["OFFSOURCE"].shape[1:] 

model = create_gabbard(input_shape_onsource, input_shape_offsource)

# Now you can print the model summary
model.summary()

# Model compilation
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Or any other loss function appropriate for your task
    metrics=['accuracy']
)

training_dataset = AdapterDataset(training_dataset)
validation_dataset = AdapterDataset(validation_dataset)
testing_dataset = AdapterDataset(testing_dataset)

2025-12-14 01:49:35,081 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025-12-14 01:49:35,621 - INFO - Loading event times from cache.
2025-12-14 01:49:36,729 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025-12-14 01:49:37,254 - INFO - Loading event times from cache.
2025-12-14 01:49:37,450 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025-12-14 01:49:37,978 - INFO - Loading event times from cache.
2025-12-14 01:49:38,074 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1243715584.1, 1243717357.9), ext='gwf', urltype='file', match=None
2025-12-14 01:49:38,082 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e90

Model: "custom"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ONSOURCE            │ (None, 1, 4096)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ OFFSOURCE           │ (None, 1, 32768)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ whiten (Whiten)     │ (None, 1, 2048)   │          0 │ ONSOURCE[0][0],   │
│                     │                   │            │ OFFSOURCE[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute (Permute)   │ (None, 2048, 1)   │          0 │ whiten[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Convolutional_1     │ (None, 1985, 8)   │        520 │ permute[0][0]     │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ELU_1 (ELU)         │ (None, 1985, 8)   │          0 │ Convolutional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Pooling_1           │ (None, 496, 8)    │          0 │ ELU_1[0][0]       │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Convolutional_2     │ (None, 465, 8)    │      2,056 │ Pooling_1[0][0]   │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ELU_2 (ELU)         │ (None, 465, 8)    │          0 │ Convolutional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Convolutional_3     │ (None, 434, 16)   │      4,112 │ ELU_2[0][0]       │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ELU_3 (ELU)         │ (None, 434, 16)   │          0 │ Convolutional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Pooling_3           │ (None, 108, 16)   │          0 │ ELU_3[0][0]       │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Flatten (Flatten)   │ (None, 1728)      │          0 │ Pooling_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_1 (Dense)     │ (None, 64)        │    110,656 │ Flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ELU_7 (ELU)         │ (None, 64)        │          0 │ Dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dropout_1 (Dropout) │ (None, 64)        │          0 │ ELU_7[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_2 (Dense)     │ (None, 64)        │      4,160 │ Dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ELU_8 (ELU)         │ (None, 64)        │          0 │ Dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dropout_2 (Dropout) │ (None, 64)        │          0 │ ELU_8[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ INJECTION_MASKS     │ (None, 1)         │         65 │ Dropout_2[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 121,569 (474.88 KB)

 Trainable params: 121,569 (474.88 KB)

 Non-trainable params: 0 (0.00 B)

If the previous cell failed it is most likely because you attempted to run it twice within the same kernel session. The kernal must be restarted in order to generate a fresh TensorFlow stratergy and recompile the model.

Finally, we can train the model with our generated dataset:

In [5]:

history = model.fit(
    training_dataset,
    epochs=10,  # Number of epochs to train for
    validation_data=validation_dataset,
)

Epoch 1/10
 250/1000 ━━━━━━━━━━━━━━━━━━━━ 37s 51ms/step - accuracy: 0.6039 - loss: 0.6875

2025-12-14 01:50:19,499 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1267558400.1, 1267560173.9), ext='gwf', urltype='file', match=None


 252/1000 ━━━━━━━━━━━━━━━━━━━━ 37s 51ms/step - accuracy: 0.6049 - loss: 0.6860

2025-12-14 01:50:19,508 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000


 506/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 64ms/step - accuracy: 0.6993 - loss: 0.5508

...

 948/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.9927 - loss: 0.0295

2025-12-14 02:14:04,832 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000


 999/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.9927 - loss: 0.0297

2025-12-14 02:14:22,720 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1241985024.1, 1241986797.9), ext='gwf', urltype='file', match=None
2025-12-14 02:14:22,728 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025-12-14 02:14:42,228 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1243688960.1, 1243690733.9), ext='gwf', urltype='file', match=None
2025-12-14 02:14:42,235 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025-12-14 02:15:01,910 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1261854720.1, 1261855793.9), ext='gwf', urltype='file', match=None
2025-12-14 02:15:01,918 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 150s 150ms/step - accuracy: 0.9923 - loss: 0.0338 - val_accuracy: 0.9870 - val_loss: 0.0452


In [6]:
model.evaluate(
    testing_dataset, 
)

2025-12-14 02:15:26,593 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1247954944.1, 1247955377.9), ext='gwf', urltype='file', match=None
2025-12-14 02:15:26,601 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000


  62/1000 ━━━━━━━━━━━━━━━━━━━━ 46s 50ms/step - accuracy: 0.9898 - loss: 0.0268

2025-12-14 02:15:31,594 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1260365824.1, 1260367597.9), ext='gwf', urltype='file', match=None
2025-12-14 02:15:31,601 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000


 318/1000 ━━━━━━━━━━━━━━━━━━━━ 49s 73ms/step - accuracy: 0.9866 - loss: 0.0539

2025-12-14 02:15:51,669 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1263588386.1, 1263589101.9), ext='gwf', urltype='file', match=None


...

2025-12-14 02:16:21,240 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1266784256.1, 1266786029.9), ext='gwf', urltype='file', match=None
2025-12-14 02:16:21,249 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000


 958/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - accuracy: 0.9869 - loss: 0.0519

2025-12-14 02:16:40,963 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1253785600.1, 1253787373.9), ext='gwf', urltype='file', match=None
2025-12-14 02:16:40,970 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 83s 81ms/step - accuracy: 0.9877 - loss: 0.0472


[0.04718460142612457, 0.98765629529953]